<a href="https://colab.research.google.com/github/flakesss/TUBES_AI/blob/hatta/tubes_ai_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
                             confusion_matrix, classification_report, roc_curve, auc)
import xgboost as xgb
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

os.makedirs('models', exist_ok=True)
os.makedirs('visualizations', exist_ok=True)


In [ ]:
data_path = '/content/Telco Customer Churn.xlsx'

try:
    telco_data = pd.read_excel(data_path)
    print(f"Data berhasil dimuat dari {data_path}")
except Exception as e:
    print(f"Error saat memuat file Excel: {e}")
    try:
        csv_path = 'Telco Customer Churn.csv'
        telco_data = pd.read_csv(csv_path)
        print(f"Data berhasil dimuat dari {csv_path}")
    except Exception as e2:
        print(f"Error saat memuat file CSV: {e2}")
        raise Exception("Tidak dapat memuat data. Pastikan file data tersedia.")

print("\nInformasi data:")
print(f"Jumlah baris: {telco_data.shape[0]}")
print(f"Jumlah kolom: {telco_data.shape[1]}")
print("\nLima baris pertama:")
print(telco_data.head())


Error saat memuat file Excel: [Errno 2] No such file or directory: '/content/Telco Customer Churn.xlsx'
Error saat memuat file CSV: [Errno 2] No such file or directory: 'Telco Customer Churn.csv'


Exception: Tidak dapat memuat data. Pastikan file data tersedia.

In [ ]:
if telco_data.shape[1] == 1 and ',' in str(telco_data.iloc[0, 0]):
    print("\nDeteksi masalah format: Data ada dalam satu kolom. Parsing data...")
    header = telco_data.columns[0].split(',')
    parsed_rows = []
    for i in range(len(telco_data)):
        values = telco_data.iloc[i, 0].split(',')
        if len(values) == len(header):
            parsed_rows.append(values)
    telco_data = pd.DataFrame(parsed_rows, columns=header)
    print("Data berhasil di-parse.")



Deteksi masalah format: Data ada dalam satu kolom. Parsing data...
Data berhasil di-parse.


In [ ]:
print("\n2. Preprocessing data...")

columns_to_drop = ['customerID']
telco_data = telco_data.drop(columns=columns_to_drop, errors='ignore')

if 'TotalCharges' in telco_data.columns:
    telco_data['TotalCharges'] = pd.to_numeric(telco_data['TotalCharges'], errors='coerce')

missing_data = telco_data.isnull().sum()
print("\nMissing values per kolom:")
print(missing_data[missing_data > 0])

num_cols = telco_data.select_dtypes(include=np.number).columns
telco_data[num_cols] = telco_data[num_cols].fillna(telco_data[num_cols].median())

print("\nDistribusi target 'Churn':")
if 'Churn' in telco_data.columns:
    print(telco_data['Churn'].value_counts())
    if telco_data['Churn'].dtype == 'object':
        telco_data['Churn'] = telco_data['Churn'].map({'Yes': 1, 'No': 0})
    print("Target 'Churn' dikonversi ke nilai biner 1/0")



2. Preprocessing data...

Missing values per kolom:
TotalCharges    11
dtype: int64

Distribusi target 'Churn':
Churn
No     5174
Yes    1869
Name: count, dtype: int64
Target 'Churn' dikonversi ke nilai biner 1/0


In [ ]:
print("\n3. Feature selection dan engineering...")

categorical_cols = telco_data.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = telco_data.select_dtypes(include=['int64', 'float64']).columns.tolist()
if 'Churn' in numerical_cols:
    numerical_cols.remove('Churn')

print(f"Kolom kategorikal: {categorical_cols}")
print(f"Kolom numerik: {numerical_cols}")

X = telco_data.drop('Churn', axis=1)
y = telco_data['Churn']



3. Feature selection dan engineering...
Kolom kategorikal: ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges']
Kolom numerik: ['TotalCharges']


In [ ]:
print("\n4. Membagi data menjadi train, validation, dan test...")

X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val
)

print(f"Jumlah data training: {X_train.shape[0]} ({X_train.shape[0]/len(telco_data):.1%} dari total)")
print(f"Jumlah data validasi: {X_val.shape[0]} ({X_val.shape[0]/len(telco_data):.1%} dari total)")
print(f"Jumlah data testing: {X_test.shape[0]} ({X_test.shape[0]/len(telco_data):.1%} dari total)")



4. Membagi data menjadi train, validation, dan test...
Jumlah data training: 4225 (60.0% dari total)
Jumlah data validasi: 1409 (20.0% dari total)
Jumlah data testing: 1409 (20.0% dari total)


In [ ]:
print("\n5. Membuat preprocessing pipeline...")

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])



5. Membuat preprocessing pipeline...


In [ ]:
print("\n6. Melatih model XGBoost dengan parameter yang lebih baik...")

xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    random_state=42,
    n_estimators=500,
    max_depth=4,
    min_child_weight=3,
    gamma=0.25,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=3
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb_model)
])

pipeline.fit(X_train, y_train)

y_pred_val = pipeline.predict(X_val)
y_pred_prob_val = pipeline.predict_proba(X_val)[:, 1]

print("\nEvaluasi model dasar pada data validasi:")
print(f"Accuracy: {accuracy_score(y_val, y_pred_val):.4f}")
print(f"Precision: {precision_score(y_val, y_pred_val):.4f}")
print(f"Recall: {recall_score(y_val, y_pred_val):.4f}")
print(f"F1-score: {f1_score(y_val, y_pred_val):.4f}")
print(f"ROC AUC: {roc_auc_score(y_val, y_pred_prob_val):.4f}")



6. Melatih model XGBoost dengan parameter yang lebih baik...

Evaluasi model dasar pada data validasi:
Accuracy: 0.7339
Precision: 0.4990
Recall: 0.6604
F1-score: 0.5685
ROC AUC: 0.7940


In [ ]:
cm = confusion_matrix(y_val, y_pred_val)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Model Dasar (Validasi)')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.savefig('visualizations/base_model_confusion_matrix.png')
plt.close()

fpr, tpr, _ = roc_curve(y_val, y_pred_prob_val)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Model Dasar (Validasi)')
plt.legend(loc="lower right")
plt.savefig('visualizations/base_model_roc_curve.png')
plt.close()


In [ ]:
print("\n7. Melakukan hyperparameter tuning dengan RandomizedSearchCV...")
param_dist = {
    'classifier__max_depth': [3, 4, 5, 6],
    'classifier__min_child_weight': [1, 3, 5, 7],
    'classifier__gamma': [0, 0.1, 0.2, 0.4],
    'classifier__subsample': [0.6, 0.8, 1.0],
    'classifier__colsample_bytree': [0.6, 0.8, 1.0],
    'classifier__learning_rate': [0.01, 0.05, 0.1]
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=50,
    cv=cv,
    scoring='roc_auc',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

random_search.fit(X_train_val, y_train_val)

print("\nParameter optimal:")
print(random_search.best_params_)
print(f"\nBest score: {random_search.best_score_:.4f}")




7. Melakukan hyperparameter tuning dengan RandomizedSearchCV...
Fitting 3 folds for each of 50 candidates, totalling 150 fits

Parameter optimal:
{'classifier__subsample': 0.6, 'classifier__min_child_weight': 7, 'classifier__max_depth': 3, 'classifier__learning_rate': 0.01, 'classifier__gamma': 0.2, 'classifier__colsample_bytree': 0.6}

Best score: 0.8495


In [ ]:
print("\n8. Evaluasi model final pada data test...")

best_model = random_search.best_estimator_


y_pred_test = best_model.predict(X_test)
y_pred_prob_test = best_model.predict_proba(X_test)[:, 1]

print("\nEvaluasi model final pada data test:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_test):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_test):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_test):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_test):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_pred_prob_test):.4f}")

print("\nClassification report:")
print(classification_report(y_test, y_pred_test, digits=4))

cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Model Final (Test)')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.savefig('visualizations/final_model_confusion_matrix.png')
plt.close()

fpr, tpr, _ = roc_curve(y_test, y_pred_prob_test)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Model Final (Test)')
plt.legend(loc="lower right")
plt.savefig('visualizations/final_model_roc_curve.png')
plt.close()



8. Evaluasi model final pada data test...


NameError: name 'random_search' is not defined

In [ ]:
print("\n9. Analisis feature importance...")

xgb_model = best_model.named_steps['classifier']
preprocessor = best_model.named_steps['preprocessor']
cat_features = preprocessor.transformers_[1][1].get_feature_names_out(categorical_cols)
feature_names = numerical_cols + list(cat_features)

importance = xgb_model.feature_importances_
indices = np.argsort(importance)[::-1]

plt.figure(figsize=(12, 8))
plt.title('Feature Importance')
plt.bar(range(len(importance)), importance[indices])
plt.xticks(range(min(20, len(importance))), [feature_names[i] for i in indices[:20]], rotation=90)
plt.tight_layout()
plt.savefig('visualizations/feature_importance.png')
plt.close()

feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)
print("\nTop 10 fitur paling penting:")
print(feature_importance_df.head(10))

feature_importance_df.to_csv('visualizations/feature_importance.csv', index=False)
joblib.dump(best_model, 'models/telco_churn_xgboost.pkl')
print("Model disimpan sebagai: models/telco_churn_xgboost.pkl")
print("\nPelatihan model selesai!")
print("Visualisasi disimpan di folder 'visualizations'")
